### ⬇️ Imports & Configs
**⚠️ IMPORTANT: Run this before anything else**

*(You can change the numbers within CONFIG)*

In [20]:

import os
import cv2
import numpy as np
import tensorflow as tf
from PIL import Image, ImageOps
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
import datetime
import threading

CONFIG = {
    'IMG_SIZE': (224, 224), # Max: 299x299 for InceptionV3
    'BATCH_SIZE': 24, # ~24 for Laptop, ~48 for PC
    'EPOCHS': 3, # I tested 5 is seems to be quite gd, but increase if you're willing to train for longer
    'DISTRIBUTE_CLASS_WEIGHTS': True # Default: True, will alleviate uneven class image distribution
}

data_dir = './data'
class_names = ['Finished', 'Opened', 'Sealed']

print("✅ Done!")

✅ Done!


# 1. Model Training
**Trains the model then saves it as a .keras model file.**

In [ ]:
def check_dataset_balance(data_dir):
    """Verify class distribution before training."""
    counts = {}
    for class_dir in sorted(os.listdir(data_dir)):  # Consistent order
        class_path = os.path.join(data_dir, class_dir)
        if os.path.isdir(class_path):
            counts[class_dir] = len([f for f in os.listdir(class_path) 
                                   if f.lower().endswith(('.jpg', '.png', '.jpeg'))])
    
    total = sum(counts.values())
    print("Dataset Balance:")
    print(counts)
    print("Ratios:", {k: f"{v/total*100:.1f}%" for k,v in counts.items()})
    return counts

def get_image_generators(data_dir, img_size, batch_size):
    """Simplified 80/10/10 split - more reliable than validation_split trick."""
    seed = 42
    
    # Train: augmentation
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        validation_split=0.2  # 80/20 split
    )
    
    train_gen = train_datagen.flow_from_directory(
        data_dir, target_size=img_size, batch_size=batch_size,
        class_mode='categorical', subset='training', seed=seed
    )
    
    # Val/Test: no augmentation
    val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.125)  # 10% val, 10% test
    
    val_gen = val_datagen.flow_from_directory(
        data_dir, target_size=img_size, batch_size=batch_size,
        class_mode='categorical', subset='training', seed=seed, shuffle=False
    )
    
    test_gen = val_datagen.flow_from_directory(
        data_dir, target_size=img_size, batch_size=batch_size,
        class_mode='categorical', subset='validation', seed=seed, shuffle=False
    )
    
    return train_gen, val_gen, test_gen

def build_improved_model(img_shape, num_classes):
    """Better architecture: GlobalAvgPool2D + more layers."""
    base_model = InceptionV3(input_shape=(*img_shape, 3), include_top=False, weights='imagenet')
    
    # Freeze base
    base_model.trainable = False
    
    # Better head (no Flatten = less params)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.3)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

# MAIN TRAINING PIPELINE
print("🔍 Checking dataset...")
check_dataset_balance(data_dir)

print("\n📊 Loading data...")
train_gen, val_gen, test_gen = get_image_generators(
    data_dir, CONFIG['IMG_SIZE'], CONFIG['BATCH_SIZE']
)

print("\n🏗️ Building model...")
model = build_improved_model(CONFIG['IMG_SIZE'], len(class_names))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
# model.summary()

print("\n⚖️ Computing class weights...")
class_weights = compute_class_weight(
    'balanced', classes=np.unique(train_gen.classes), y=train_gen.classes
)
class_indices = train_gen.class_indices  # {'Finished': 0, 'Opened': 1, 'Sealed': 2}
class_weight_dict = {class_indices[class_name]: weight 
                     for class_name, weight in zip(class_indices.keys(), class_weights)}
print("Class weights:", class_weight_dict)

# Callbacks
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True, monitor='val_accuracy'),
    ReduceLROnPlateau(factor=0.2, patience=3, min_lr=1e-7)
]

print("\n🚀 Training...")
history = model.fit(
    train_gen,
    epochs=CONFIG['EPOCHS'],
    validation_data=val_gen,
    class_weight=class_weight_dict,  # Handles your imbalance!
    callbacks=callbacks,
    verbose=1
)

print("\n✅ Test accuracy:", model.evaluate(test_gen)[1])

# Save with timestamp
timestamp = datetime.datetime.now().strftime("%b%d_%H%M")
model.save(f'./models/cup_noodle_classifier_{timestamp}.keras')
print(f"💾 Saved: cup_noodle_classifier_{timestamp}.keras")

🔍 Checking dataset...
Dataset Balance:
{'Finished': 548, 'Opened': 448, 'Sealed': 675}
Ratios: {'Finished': '32.8%', 'Opened': '26.8%', 'Sealed': '40.4%'}

📊 Loading data...
Found 1338 images belonging to 3 classes.
Found 1463 images belonging to 3 classes.
Found 208 images belonging to 3 classes.

🏗️ Building model...

⚖️ Computing class weights...
Class weights: {0: np.float64(1.0159453302961277), 1: np.float64(1.2423398328690807), 2: np.float64(0.825925925925926)}

🚀 Training...
Epoch 1/3
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6157 - loss: 1.2759

## ======================================
# 2. Trying the Model

### 🔃 Load a .keras model
Loading a model is necessary if the above code to train a new model was not executed.
(Skip this if model was recently trained above)

In [12]:
folder = "./models/"
files = os.listdir(folder)

# sort by last added time
if not files: raise FileNotFoundError("No models found!")
files_sorted = sorted(files, key=lambda f: os.path.getmtime(os.path.join(folder, f)))
last_added_filename = files_sorted[-1]

while True:
    input_filename = input(
        f"Enter model to load (Latest: {last_added_filename.replace('.keras', '')}): "
    ).strip()

    if input_filename == "":
        model_filename = last_added_filename
        break

    model_filename = f"{input_filename.replace('.keras', '')}.keras"
    full_path = os.path.join(folder, model_filename)

    if os.path.exists(full_path):
        break
    else:
        print("❌ Model not found. Please try again.")

model = tf.keras.models.load_model(f'./models/{model_filename}')

print("✅ Model Loaded:", model_filename)

✅ Model Loaded: cup_noodle_classifier_20260129_144829.keras


### new version of code below, yet to test tho

In [ ]:
import tkinter as tk
from tkinter import filedialog
import os

folder = "./models/"
files = [f for f in os.listdir(folder) if f.endswith('.keras')]

if not files:
    raise FileNotFoundError("No .keras models found in ./models/!")

# Sort by last modified (newest first)
files_sorted = sorted(files, key=lambda f: os.path.getmtime(os.path.join(folder, f)), reverse=True)
latest_model = files_sorted[0]

print(f"📁 Found {len(files)} models. Latest: {latest_model}")

# File dialog - filter .keras files only
root = tk.Tk()
root.withdraw()  # Hide main window

model_paths = filedialog.askopenfilenames(
    title="Select model file (.keras)",
    initialdir=folder,
    filetypes=[("Keras models", "*.keras"), ("All files", "*.*")]
)

root.destroy()

if not model_paths:
    print("❌ No model selected. Exiting.")
    exit()

# Use first selected (or only) model
model_filename = os.path.basename(model_paths[0])
full_path = model_paths[0]

print(f"✅ Loading: {model_filename}")
model = tf.keras.models.load_model(full_path)
print("🚀 Model loaded successfully!")


### Part 2: Inference using Camera Stream 📷 
(Please view the cell under this one to terminate the camera)

In [ ]:
def import_and_predict(image_data):
    image = ImageOps.fit(image_data, CONFIG['IMG_SIZE'], Image.Resampling.LANCZOS)
    image = image.convert('RGB')
    image = np.asarray(image)
    image = (image.astype(np.float32) / 255.0)
    img_reshape = image[np.newaxis,...]
    prediction = model.predict(img_reshape, verbose=0)
    return prediction

stop_flag = False

def camera_loop():
    global stop_flag
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        cap.open()
        
    print("Camera Started!")

    while not stop_flag:
        ret, original = cap.read()
        if not ret:
            break

        image = Image.fromarray(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
        prediction = import_and_predict(image)

        idx = np.argmax(prediction)
        confidence = prediction[0][idx] 

        predict = f"It is a {class_names[idx]}! ({confidence:.2f})"

        cv2.putText(original, predict, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        cv2.imshow("Cup Noodles Detector", original)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            stop_flag = True

    cap.release()
    cv2.destroyAllWindows()

# start the camera in a separate thread
camera_thread = threading.Thread(target=camera_loop)
camera_thread.start()


Camera Started!


❌ Run to terminate the camera! 👇

In [14]:
stop_flag = True
camera_thread.join()
print("Killed the Camera!")

Killed the Camera!
